In [1]:
import os
import pandas as pd

In [4]:
def carregar_csvs_de_pasta(pasta):
    all_files = []
    for root, dirs, files in os.walk(pasta):
        for file in files:
            if file.endswith(".csv"):
                all_files.append(os.path.join(root, file))
    
    df_list = []
    for file in all_files:
        try:
            df = pd.read_csv(file, sep=';', encoding='latin1')
            df_list.append(df)
        except Exception as e:
            print(f"Erro ao ler {file}: {e}")
    
    if df_list:
        return pd.concat(df_list, ignore_index=True)
    else:
        return pd.DataFrame()

In [5]:
# Defina os caminhos das pastas contendo os CSVs
pasta_bens = 'C:\\Projetos\\Semestre 8\\CienciaDeDados\\n2\\bem candidato\\'
pasta_cons_cand = 'C:\\Projetos\\Semestre 8\\CienciaDeDados\\n2\\consulta candidato\\'


In [6]:
# Carregar todos os CSVs das pastas
df_bens = carregar_csvs_de_pasta(pasta_bens)
df_bens['VR_BEM_CANDIDATO'] = df_bens['VR_BEM_CANDIDATO'].str.replace('.', '', regex=False).str.replace(',', '.', regex=False)

df_candidatos = carregar_csvs_de_pasta(pasta_cons_cand)

# Filtrar prefeitos eleitos em 2024
prefeitos_2024 = df_candidatos[(df_candidatos['DS_CARGO'] == 'PREFEITO') & (df_candidatos['ANO_ELEICAO'] == 2024)]
prefeitos_eleitos_2024 = prefeitos_2024[prefeitos_2024['DS_SIT_TOT_TURNO'] == 'ELEITO']

# Mesclar as tabelas de bens e candidatos
candidatos_bens = pd.merge(prefeitos_eleitos_2024, df_bens, on='SQ_CANDIDATO')

# Converter valores dos bens em float
candidatos_bens['VR_BEM_CANDIDATO'] = candidatos_bens['VR_BEM_CANDIDATO'].replace(',', '.', regex=True).astype(float)

# Agrupar e encontrar o candidato com mais bens
candidato_com_mais_bens = candidatos_bens.groupby(['SQ_CANDIDATO', 'NM_CANDIDATO'])['VR_BEM_CANDIDATO'].sum().reset_index()
candidato_com_mais_bens = candidato_com_mais_bens.sort_values(by='VR_BEM_CANDIDATO', ascending=False)

# Exibir o candidato com maior valor de bens
candidato_elegido = candidato_com_mais_bens.head(1)

# Exibindo o insight
print(f"O candidato eleito para prefeito em 2024 que declarou a maior quantidade de bens foi {candidato_elegido['NM_CANDIDATO'].values[0]} com um total de R$ {candidato_elegido['VR_BEM_CANDIDATO'].values[0]:,.2f}.")


O candidato eleito para prefeito em 2024 que declarou a maior quantidade de bens foi MIGUEL VAZ RIBEIRO com um total de R$ 878,863,020.80.
